## Temporal evolution of the effective network in the Deffuant opinion dynamics model

This notebook contains a detailed implementation of the simulation of the Deffuant Model on arbitrary networks.
The words network and graph is used interchangeably. Same applies to node/vertex.

#### The first chunk of code defines all the functions needed to run the simulation as well as the implmentation itself.
The implmentation relies as much as possible on vectorized operations to yield very good performance.
This is true both for the graph analysis library used called graph tool as well as the use of NumPy and SciPy.

In [ ]:
from graph_tool.generation import complete_graph
from graph_tool import Graph
import numpy as np
import copy
import time
import matplotlib.pyplot as plt # https://matplotlib.org/stable/plot_types/basic/plot.html
from scipy.sparse import random
from scipy.sparse import tril
import pickle
from tqdm import tqdm
from graph_tool.topology import label_components

def erdos_renyi(n, p, rng):
    """
    Generates an Erdos-Renyi random graph with n nodes and edge probability p
    using a NumPy random number generator.
    Returns a NumPy array representing the adjacency matrix of the graph.
    """
    adj_matrix = rng.random((n,n), dtype='float32')
    adj_matrix = np.tril(adj_matrix)
    adj_matrix += adj_matrix.T
    adj_matrix[adj_matrix > p] = 0
    adj_matrix[adj_matrix != 0] = 1
    np.fill_diagonal(adj_matrix, 0)
    return adj_matrix.astype("bool")


def erdos_renyi_scipy(n, p, rng):
    """
    Generates an Erdos-Renyi random graph with n nodes and edge probability p
    using a NumPy random number generator.
    Returns a SciPy sparse matrix representing the adjacency matrix of the graph.
    """
    adj_matrix = random(n, n, density=p, format='csr', dtype="bool", random_state=rng, data_rvs=np.ones)
    adj_matrix = tril(adj_matrix)
    adj_matrix = adj_matrix + adj_matrix.T
    adj_matrix.setdiag(0)
    return adj_matrix.astype('bool').toarray()

def adj_matrix_to_edge_list(adj_matrix):
    """
    Transforms an adjacency matrix of an undirected graph into a list of edges
    """
    edge_list = np.column_stack(np.where(np.tril(adj_matrix) == 1))
    return edge_list

def erdos_renyi_graph(n, p, rng):
    """
    Generates an Erdos-Renyi random graph with n nodes and edge probability p
    using a NumPy random number generator.
    Returns the Erdos-Renyi random graph as a graph_tool Graph.
    """
    e_list = adj_matrix_to_edge_list(erdos_renyi(n=n, p=p, rng=rng))
    g = Graph(directed=False)
    g.add_edge_list(e_list)
    return g

def get_random_node_uniform(rng, a_huge_key_list):
    """
    Returns a random uniform entry from a list
    """
    L = len(a_huge_key_list)
    i = rng.integers(0, L)
    return a_huge_key_list[i]


def get_net(net_type, net_order, rng=np.random.default_rng(42), p = None):
    """
    Generates a network of the desired type

    Parameters
    ----------
    net_type : str
        The desired graph. Currently supports complete graph and erdos renyi random graph
    net_order : int
        the size of the network
    rng : np.random.Generator
        A NumPy random number generator
    p : float
        In the case of erdos renyi graph, the desired edge probability.
        If none is specified p is chosen to yield a connected graph 
        with high proability
          
    Returns
    -------
    graph_tool.Graph
        A graph of the specified type
    """
    if net_type == 'complete_mixing':
        return complete_graph(N=net_order)

    elif net_type == 'erdos':
        if p == None:
            p = ((1+0.001)*np.log(net_order))/net_order 
            p += 0.001
            return erdos_renyi_graph(net_order, p, rng)
        return erdos_renyi_graph(net_order, p, rng)
    else:
        return None

# return all the discussed metrics
def metrics(G, components, eff_edges):
    """
    Computes percolation metrics by determing the connected 
    components of a graph.

    Parameters
    ----------
    G : graph_tool.Graph
        The graph on which to compute the metrics.
    components : graph_tool.VertexPropertyMap 
        the connected components at the previous time step
    eff_edges : graph_tool.EdgePropertyMap
        A boolean array of edges contained in the effective network.    

    Returns
    -------
    graph_tool.VertexPropertyMap 
        the connected components at the current time step
    float
        Fractional size of the largest component
    float
        Average squared size of non-giant components
    """
    components, sizes = label_components(G, vprop = components, directed = False)
    if len(sizes) == 1:
        return components, 1, 1, 0
    mask = np.ones(sizes.size, dtype=bool)
    mask[sizes.argmax()] = False
    return components, sizes.max()/sizes.sum(), (sizes[mask]**2).mean()


def update_opinions(u, v, opinions, convergence, symmetric_updating=True):
    """
    Updates the opinion of nodes

    Parameters
    ----------
    u : int
        The node id of u
    v : int
        The node id of u's neighbor v
    opinions : graph_tool.VertexPropertyMap
        An array of opinions.
    convergence : float
        The value used for updating opinions in the model.
    symmetric_updating: bool
        If True, at every timestep both nodes update their opinions.
        If False, at every timestep only the first selected node updates its opinion.
    

    Returns
    -------
    np.array
        An array masking nodes outside the confidence bound of u
    """
    #we need to remember to access the array using .a
    diff = opinions.a[u] - opinions.a[v]

    opinions.a[u] -= convergence * diff
    
    if symmetric_updating:
        opinions.a[v] += convergence * diff
        
    return opinions.a


def effective_neighbors(u, neigh_u, opinions, threshold):
    """
    Finds the neighbors of a node with opinion within confidence bound

    Parameters
    ----------
    u : int
        The node id of u
    neigh_u : np.array
        An array of edges from u to all its neighbors with edge ids
    opinions : graph_tool.VertexPropertyMap
        An array of opinions.
    threshold : float
        Threshold value for node interaction in the model. 

    Returns
    -------
    np.array
        An array masking nodes outside the confidence bound of u
    """

    # neigh_u first column contains u, second neighbor v, third edge index
    #compute the updates by indexing the neighbor nodes from opinions and subtracting opinion of u
    #this means the len of distances equals number of neighbors of u
    distances = opinions.a[neigh_u[:,1]] - opinions.a[u]
    
    return np.where((distances < -threshold) | (distances > threshold), 0, 1) # filters out non-neighbors with close opinion distance


def update_effective_net(u, v, neigh_u, neigh_v, eff_edges, opinions, threshold, symmetric_updating):
    """
    Updates the array of edges in the effective network

    Parameters
    ----------
    u : int
        The node id of u
    v : int
        The node id of u's neighbor v
    neigh_u : np.array
        An array of edges from u to all its neighbors with edge ids
    neigh_v : np.array
        An array of edges from v to all its neighbors with edge ids
    eff_edges : graph_tool.EdgePropertyMap
        A boolean array of edges contained in the effective network.
    threshold : float
        Threshold value for node interaction in the model. 
    symmetric_updating: bool
        If True, Noth nodes update their opinions.
        If False, only the first selected node updates its opinion.
   


    Returns
    -------
    eff_edges : graph_tool.EdgePropertyMap
        A boolean array of edges contained in the effective network.
    """
    # this gets updates to the effective edges, neighbors has len equalt to number of u's neighbors
    neighbors = effective_neighbors(u, neigh_u, opinions, threshold)

    #index from the effective edges the edges that form neighbors with u using edge id stored in neigh_u
    eff_edges.a[neigh_u[:,-1]]   = neighbors
        
    if not symmetric_updating:
        return eff_edges.a

    neighbors = effective_neighbors(v, neigh_v, opinions, threshold)
    
    eff_edges.a[neigh_v[:,-1]] = neighbors
            
    return eff_edges.a


def simulate_deffuant_model(G: Graph, iterations: int, threshold: float, convergence: float, initial_opinions=None,\
                            symmetric_updating=True, fast_mode=False, rng=np.random.default_rng(42), offset = -1, early_return = False, \
                                check = 1000, res_threshold = 0.001):
    """
    Simulates the Deffuant model on the given graph.

    Parameters
    ----------
    G : graph_tool.Graph
        The graph on which to simulate the model.
    iterations : int
        The  maximum number of iterations to run the simulation for.
    threshold : float
        Threshold value for node interaction in the model.
    convergence : float
        The value used for updating opinions in the model.
    initial_opinions : np.array
        A user specified list of opinions.
    symmetric_updating: bool
        If True, at every timestep both nodes update their opinions.
        If False, at every timestep only the first selected node updates its opinion.
    fast_mode: bool
        If True, only the metrics of the final effective network is returned and used throughout the simulation.
        If False, the metrics of the effective network is saved and returned for every time step.
    rng : np.random.Generator
        A numpy random number generator.
    offset : int
        A value to specify the first n numbers are run in fast mode.
    early_return : bool
        If True, returns the effective network at time step 0
    check : int
        Check if opinions have converged every K time steps.
    res_threshold : float
        The resolution threshold to compare the mean absolute difference in opinions between checks.  



    Returns
    -------
    graph_tool.Graph
        A graphs, where each graph represents the state of the network at the final step.
    graph_tool.VertexPropertyMap
        An array of the final opinions
    If fast mode:
    graph_tool.VertexPropertyMap
        the connected components at the final time step
    float
        Fractional size of the largest component
    float
        Average squared size of non-giant components
    [float]
        A list of the mean absolute difference in opinion.
    Else:
    [graph_tool.VertexPropertyMap]
        A list of the connected components at every time step
    [float]
        A list of the fractional size of the largest component at every time step
    [float]
        A list of the Average squared size of non-giant components  at every time step
    [float]
        A list of the mean absolute difference in opinion
    """

    #Copy the input graph to create the effective graph on which we will compute connected components 
    G_eff = copy.deepcopy(G)

    #Create numpy like arrays 
    #to store intermediate components
    components = G_eff.new_vertex_property('int')
    #G_eff.vertex_properties['comp'] = components

    # initialize opinions with another numpy like array
    opinions = G.new_vertex_property('double')
    #G_eff.vertex_properties['ops'] = opinions
    
    #to store effective edges
    eff_edges = G_eff.new_edge_property('bool')
    eff_edges.a = True
    #apply the effective edges as a filter
    G_eff.set_edge_filter(eff_edges)

    
    if initial_opinions is None:
        # .a is used to get the array
        opinions.a = rng.random(G.num_vertices(ignore_filter=True))
    else:
        opinions.a = np.copy(initial_opinions)

    for u in G.iter_vertices():
        # get all the edges formin neighbors with u
        # by setting [G.edge_index] we also get edge ids
        # neigh_u first column contains u, second neighbor v, third edge index
        neigh_u = G.get_all_edges(u, [G.edge_index])
        neighbors = effective_neighbors(u, neigh_u, opinions, threshold)
        eff_edges.a[neigh_u[:,-1]]   = neighbors
    
    #return effective network network after initial setup
    if early_return:
        return G_eff

    if not fast_mode:
        # initialize the list results that will be returned
        components, dispersion, c_1, avg_sq_clust = metrics(G_eff, components, eff_edges)
        dispersions = [dispersion]
        c_1s = [c_1]
        avg_sq_clusts = [avg_sq_clust]

    #Initialize array of previous opinions
    opinions_prev = np.copy(opinions.a)
    mean_opinion_diff = 0
    #Initialize list of mean difference in opinions
    opp_diffs = [mean_opinion_diff]

    consecutive_checks = 0
    # run simulation
    for t in np.arange(1, iterations):
        
        # choose random node to update
        u = rng.integers(0, G.num_vertices(ignore_filter=True))
        # by setting [eff_edges, G.edge_index] we also get whether edgeis in effective network and edge ids
        # neigh_u first column contains u, second neighbors v, third contains a boolean indicating whether
        # edge is in effective network, fourth contains edge index
        neigh_u = G.get_all_edges(u, [eff_edges, G.edge_index])
        
        #Check if u is isolated
        while not np.any(neigh_u[:,2]):
            u = rng.integers(0, G.num_vertices(ignore_filter=True))
            neigh_u = G.get_all_edges(u, [eff_edges, G.edge_index])
        
            

        # get a random neighbor of u by chosing from the neighbors that have effective edges
        v = rng.choice(neigh_u[neigh_u[:,2]==1,1])
        # we don't need effective edges for v
        neigh_v = G.get_all_edges(v, [G.edge_index])
        
        # update their opinions
        opinions.a = update_opinions(u, v, opinions, convergence, symmetric_updating)
        
        if t % check == 0:
            #print(f'checking {t}')
            mean_opinion_diff = np.abs(opinions_prev-opinions.a).mean()
            opp_diffs.append(mean_opinion_diff)
            opinions_prev = np.copy(opinions.a)
            if mean_opinion_diff < res_threshold:
                consecutive_checks += 1
                if consecutive_checks >= 2:
                    #print(f'checking {t}')
                    break
            else:
                consecutive_checks = 0

        
        # we now update the connections of these nodes in the effective network depending on
        # the opinions of their neighbors in the underlying network G
        eff_edges.a = update_effective_net(u, v, neigh_u, neigh_v, eff_edges, opinions, threshold, symmetric_updating)

        #offset from fast mode is available to skip connected component on first t trials
        if not fast_mode and offset<t:
            # add it to list

            components, dispersion, c_1, avg_sq_clust = metrics(G_eff, components, eff_edges)
            dispersions.append(dispersion)
            c_1s.append(c_1)
            avg_sq_clusts.append(avg_sq_clust)
    
    
    if fast_mode:
        return G_eff, opinions, *metrics(G_eff, components, eff_edges), opp_diffs
    else:
        return G_eff, opinions, components, c_1s, avg_sq_clusts, opp_diffs

#### The following code block contains the code for running the simulations across a grid of parameter values 

In [ ]:
# FUNCTIONS for simulating data for plotting

# computes the avg value of a metric over a range of d values in a given net type and fixed convergence
def metric_d_data(net_type, net_order, symmetric_updating, iterations,\
                       sample_size, num_points, init_d, final_d, convergence, rng, check = 1000, res_threshold = 0.001, p = None):
    """
    Simulates the Deffuant model on the given graph using grid search over values of d.
    The function computes the evolution in percolation metrics for a number of d values
    using a set sample size.

    Parameters
    ----------
    net_type : str
        A string specifying which model to simulate.
    net_order : int
        the size of the network
    symmetric_updating: bool
        If True, at every timestep both nodes update their opinions.
        If False, at every timestep only the first selected node updates its opinion.
    iterations : int
        The maximum number of iterations to run the simulation for.
    sample_size : int
        The number of points to sample for each value of d
    num_points : int
        The number of values used from the linear grid
    init_d : int
        The first d (threshold | confidence bound) value in the grid
    final_d : int
        The last d value (threshold | confidence bound) in the grid (not included)
    convergence : float
        The value used for updating opinions in the model.
    rng : np.random.Generator
        A numpy random number generator.
    check : int
        Check if opinions have converged every K time steps.
    res_threshold : float
        The resolution threshold to compare the mean absolute difference in opinions between checks.  
    p : float
        In the case of erdos renyi graph, the desired edge probability.
        If none is specified p is chosen to yield a connected graph 
        with high proability


    Returns
    -------
    [np.array float]
        A list of arrays of arrays of the mean of the fractional size of the largest component using parameter d from n samples
    [np.array float]
        A list of arrays of arrays of the variance of the fractional size of the largest component using parameter d from n samples
    [np.array float]
        A list of arrays of the mean of the Average squared size of non-giant components using parameter d from n samples
    [np.array float]
        A list of arrays of the variance of the Average squared size of non-giant components using parameter d from n samples
    [float]
        The grid of d values
    """
    # initialize vector with data
    c_1_avgs = []
    x_hat_avgs = []
    c_1_vars = []
    x_hat_vars = []
    # initialize vector with x-axis points
    d = [init_d + i*(final_d-init_d)/num_points for i in range(num_points)]
    
    for i in tqdm(range(num_points), desc="number of points"):
        #print(".", end='') # loading
        c_1 = []
        x_hat = []
        # at every point we will average the metric over sample_size networks
        for j in tqdm(range(sample_size),desc="samples"):
            
            # generate network sample
            initial_net = get_net(net_type, net_order, rng, p)

            g, ops, comps, C, X, op_dif = simulate_deffuant_model(initial_net, iterations, d[i], convergence,
                                                 symmetric_updating=symmetric_updating, fast_mode=False, rng=rng, check = 1000, res_threshold = 0.001)
            
            # we pad the arrays with the metric values at convergence
            C = np.pad(C, (0, iterations-len(X)), 'edge')
            X = np.pad(X, (0, iterations-len(X)), 'edge')
            c_1.append(C)
            x_hat.append(X)

        c_1 = np.array(c_1)
        x_hat = np.array(x_hat)

        c_1_avgs.append(c_1.mean(axis=0))
        x_hat_avgs.append(x_hat.mean(axis=0))
        c_1_vars.append(c_1.var(axis=0))
        x_hat_vars.append(x_hat.var(axis=0))
    print()

    return c_1_avgs, c_1_vars, x_hat_avgs, x_hat_vars, d


# computes the avg value of a metric over a range of d values in a given net type and fixed convergence
def metric_mu_data(net_type, net_order, symmetric_updating, iterations,\
                       sample_size, num_points, d, init_mu, final_mu, rng, check = 1000, res_threshold = 0.001, p = None):
    """
    Simulates the Deffuant model on the given graph using grid search over values of d.
    The function computes the evolution in percolation metrics for a number of d values
    using a set sample size.

    Parameters
    ----------
    net_type : str
        A string specifying which model to simulate.
    net_order : int
        the size of the network
    symmetric_updating: bool
        If True, at every timestep both nodes update their opinions.
        If False, at every timestep only the first selected node updates its opinion.
    iterations : int
        The maximum number of iterations to run the simulation for.
    sample_size : int
        The number of points to sample for each value of mu
    num_points : int
        The number of values used from the linear grid
    d : float
        Threshold value for node interaction in the model.
    init_mu : int
        The first mu (convergence) value in the grid
    final_mu : int
        The last d value (convergence) in the grid (not included)
    rng : np.random.Generator
        A numpy random number generator.
    check : int
        Check if opinions have converged every K time steps.
    res_threshold : float
        The resolution threshold to compare the mean absolute difference in opinions between checks.  
    p : float
        In the case of erdos renyi graph, the desired edge probability.
        If none is specified p is chosen to yield a connected graph 
        with high proability


    Returns
    -------
    [np.array float]
        A list of arrays of arrays of the mean of the fractional size of the largest component using parameter mu from n samples
    [np.array float]
        A list of arrays of arrays of the variance of the fractional size of the largest component using parameter mu from n samples
    [np.array float]
        A list of arrays of the mean of the Average squared size of non-giant components using parameter mu from n samples
    [np.array float]
        A list of arrays of the variance of the Average squared size of non-giant components using parameter mu from n samples
    [float]
        The grid of mu values
    """
    # initialize vector with data
    c_1_avgs = []
    x_hat_avgs = []
    c_1_vars = []
    x_hat_vars = []
    # initialize vector with x-axis points
    mu = [init_mu + i*(final_mu-init_mu)/num_points for i in range(num_points)]
    
    for i in tqdm(range(num_points), desc="number of points"):
        #print(".", end='') # loading
        c_1 = []
        x_hat = []
        # at every point we will average the metric over sample_size networks
        for j in tqdm(range(sample_size),desc="samples"):
            
            # generate network sample
            initial_net = get_net(net_type, net_order, rng, p)

            g, ops, comps, C, X, op_dif = simulate_deffuant_model(initial_net, iterations, d, mu[i],
                                                 symmetric_updating=symmetric_updating, fast_mode=False, rng=rng, check = 1000, res_threshold = 0.001)
            
            # we pad the arrays with the metric values at convergence
            C = np.pad(C, (0, iterations-len(X)), 'edge')
            X = np.pad(X, (0, iterations-len(X)), 'edge')
            c_1.append(C)
            x_hat.append(X)

        c_1 = np.array(c_1)
        x_hat = np.array(x_hat)

        c_1_avgs.append(c_1.mean(axis=0))
        x_hat_avgs.append(x_hat.mean(axis=0))
        c_1_vars.append(c_1.var(axis=0))
        x_hat_vars.append(x_hat.var(axis=0))
    print()

    return c_1_avgs, c_1_vars, x_hat_avgs, x_hat_vars, mu

#### In total this note book simulates the Deffuant model over 8 networks

We decided to give each experiment its own code block. The simulations below take at max 3 hours. For smaller networks it is very fast.

### Simulating over a grid of d using a complete graph with 1000 nodes
We use sample size 10, for 15 points of d between 0.05 and 0.35. Convergence (mu) is fixed to 0.3

In [ ]:
rng = np.random.default_rng(42)

start = time.time()


c_1_avgs, c_1_vars, x_hat_avgs, x_hat_vars, d = metric_d_data(net_type='complete_mixing', net_order=1000, symmetric_updating=True,
                       iterations=250000, sample_size=10, num_points=15, init_d=0.05, final_d=0.35, convergence=0.3, rng=rng, p = 0.05)

total_time = round(time.time()-start, 3)
print(f"simulation time: {total_time} seconds")


res = {"c_1_avgs" : c_1_avgs, "c1_vars" : c_1_vars, "x_hat_avgs" : x_hat_avgs, "x_hat_vars" : x_hat_vars,"grid" : d }

with open('results/complete1000_d_005_035_10_15_mu_03.pickle', 'wb') as f:
    pickle.dump(res, f)


### Simulating over a grid of mu using a complete graph with 1000 nodes
We use sample size 5, for 10 points of mu between 0.05 and 0.5. Confidence threshold (d) is fixed to 0.2

In [ ]:
rng = np.random.default_rng(42)

start = time.time()

c_1_avgs, c_1_vars, x_hat_avgs, x_hat_vars, d = metric_mu_data(net_type='complete_mixing', net_order=1000, symmetric_updating=True,
                       iterations=250000, sample_size=5, num_points=10, d=0.2, init_mu=0.05, final_mu=0.5, rng=rng, p = 0.05)

total_time = round(time.time()-start, 3)
print(f"simulation time: {total_time} seconds")

res = {"c_1_avgs" : c_1_avgs, "c1_vars" : c_1_vars, "x_hat_avgs" : x_hat_avgs, "x_hat_vars" : x_hat_vars,"grid" : d }

with open('results/complete1000_mu_005_05_5_10_d_02.pickle', 'wb') as f:
    pickle.dump(res, f)


### Simulating over a grid of d using a complete graph with 100 nodes
We use sample size 20, for 20 points of d between 0.05 and 0.35. Convergence (mu) is fixed to 0.3

In [ ]:
rng = np.random.default_rng(42)

start = time.time()


c_1_avgs, c_1_vars, x_hat_avgs, x_hat_vars, d = metric_d_data(net_type='complete_mixing', net_order=100, symmetric_updating=True,
                       iterations=250000, sample_size=20, num_points=20, init_d=0.05, final_d=0.35, convergence=0.3, rng=rng, p = 0.05)

total_time = round(time.time()-start, 3)
print(f"simulation time: {total_time} seconds")


res = {"c_1_avgs" : c_1_avgs, "c1_vars" : c_1_vars, "x_hat_avgs" : x_hat_avgs, "x_hat_vars" : x_hat_vars,"grid" : d }

with open('results/complete100_d_005_035_20_20_mu_03.pickle', 'wb') as f:
    pickle.dump(res, f)


### Simulating over a grid of mu using a complete graph with 100 nodes
We use sample size 20, for 10 points of mu between 0.05 and 0.5. Confidence threshold (d) is fixed to 0.2

In [ ]:
rng = np.random.default_rng(42)

start = time.time()

c_1_avgs, c_1_vars, x_hat_avgs, x_hat_vars, d = metric_mu_data(net_type='complete_mixing', net_order=100, symmetric_updating=True,
                       iterations=250000, sample_size=20, num_points=10, d=0.2, init_mu=0.05, final_mu=0.5, rng=rng, p = 0.05)

total_time = round(time.time()-start, 3)
print(f"simulation time: {total_time} seconds")

res = {"c_1_avgs" : c_1_avgs, "c1_vars" : c_1_vars, "x_hat_avgs" : x_hat_avgs, "x_hat_vars" : x_hat_vars,"grid" : d }

with open('results/complete100_mu_005_05_20_10_d_02.pickle', 'wb') as f:
    pickle.dump(res, f)


### Simulating over a grid of d using an erdos-renyi random graph with 1000 nodes
We use sample size 10, for 15 points of d between 0.05 and 0.35. Convergence (mu) is fixed to 0.3.
We generate the random graph using edge probability 0.005 which yields connected graphs with high probability.

In [ ]:
rng = np.random.default_rng(42)

start = time.time()


c_1_avgs, c_1_vars, x_hat_avgs, x_hat_vars, d = metric_d_data(net_type='erdos', net_order=1000, symmetric_updating=True,
                       iterations=250000, sample_size=10, num_points=15, init_d=0.05, final_d=0.35, convergence=0.3, rng=rng, p = 0.005)

total_time = round(time.time()-start, 3)
print(f"simulation time: {total_time} seconds")


res = {"c_1_avgs" : c_1_avgs, "c1_vars" : c_1_vars, "x_hat_avgs" : x_hat_avgs, "x_hat_vars" : x_hat_vars,"grid" : d }

with open('results/erdos1000_d_005_035_10_15_mu_03.pickle', 'wb') as f:
    pickle.dump(res, f)


### Simulating over a grid of mu using an erdos-renyi random graph with 1000 nodes
We use sample size 10, for 20 points of mu between 0.05 and 0.5. Confidence threshold (d) is fixed to 0.15
We generate the random graph using edge probability 0.005 which yields connected graphs with high probability.

In [ ]:
rng = np.random.default_rng(42)

start = time.time()

c_1_avgs, c_1_vars, x_hat_avgs, x_hat_vars, d = metric_mu_data(net_type='erdos', net_order=1000, symmetric_updating=True,
                       iterations=250000, sample_size=10, num_points=20, d=0.15, init_mu=0.05, final_mu=0.5, rng=rng, p = 0.005)

total_time = round(time.time()-start, 3)
print(f"simulation time: {total_time} seconds")

res = {"c_1_avgs" : c_1_avgs, "c1_vars" : c_1_vars, "x_hat_avgs" : x_hat_avgs, "x_hat_vars" : x_hat_vars,"grid" : d }

with open('results/erdos1000_mu_005_05_10_20_d_015.pickle', 'wb') as f:
    pickle.dump(res, f)


### Simulating over a grid of d using an erdos-renyi random graph with 100 nodes
We use sample size 30, for 15 points of d between 0.05 and 0.35. Convergence (mu) is fixed to 0.3.
We generate the random graph using edge probability 0.05 which yields connected graphs with high probability.

In [ ]:
rng = np.random.default_rng(42)

start = time.time()


c_1_avgs, c_1_vars, x_hat_avgs, x_hat_vars, d = metric_d_data(net_type='erdos', net_order=100, symmetric_updating=True,
                       iterations=250000, sample_size=30, num_points=15, init_d=0.05, final_d=0.35, convergence=0.3, rng=rng, p = 0.05)

total_time = round(time.time()-start, 3)
print(f"simulation time: {total_time} seconds")


res = {"c_1_avgs" : c_1_avgs, "c1_vars" : c_1_vars, "x_hat_avgs" : x_hat_avgs, "x_hat_vars" : x_hat_vars,"grid" : d }

with open('results/erdos100_d_005_035_30_15_mu_03.pickle', 'wb') as f:
    pickle.dump(res, f)



### Simulating over a grid of mu using an erdos-renyi random graph with 100 nodes
We use sample size 10, for 20 points of mu between 0.05 and 0.5. Confidence threshold (d) is fixed to 0.15
We generate the random graph using edge probability 0.05 which yields connected graphs with high probability.

In [ ]:
rng = np.random.default_rng(42)

start = time.time()

c_1_avgs, c_1_vars, x_hat_avgs, x_hat_vars, d = metric_mu_data(net_type='complete_mixing', net_order=100, symmetric_updating=True,
                       iterations=250000, sample_size=20, num_points=20, d=0.2, init_mu=0.05, final_mu=0.5, rng=rng, p = 0.05)

total_time = round(time.time()-start, 3)
print(f"simulation time: {total_time} seconds")

res = {"c_1_avgs" : c_1_avgs, "c1_vars" : c_1_vars, "x_hat_avgs" : x_hat_avgs, "x_hat_vars" : x_hat_vars,"grid" : d }

with open('results/erdos100_mu_005_05_20_20_d_015.pickle', 'wb') as f:
    pickle.dump(res, f)


In [ ]:
import matplotlib.colors as mcolors
from matplotlib import cm

def get_n_colors(n):
    return cm.tab20(range(n)).tolist()

def plot_multiple_lines(lines, variances, plot_first_n = -1, title=None, xlabel=None, ylabel=None, legends = None, colors = None):
    for i, line in enumerate(lines):
        y_values = line[:plot_first_n]
        variance = variances[i][:plot_first_n]
        x_values = np.arange(len(y_values))
        if colors:
            color = colors[i]
        else:
            color = None
        plt.plot(x_values, y_values, color=color)
      # plt.fill_between(x_values, y_values-variance, y_values+variance, color=color, alpha=0.2)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend(legends, loc='center right')
    plt.show()


cols = get_n_colors(len(d))

plot_multiple_lines(c_1_avgs, c_1_vars, plot_first_n=100000, legends=[round(x, 2) for x in d], colors=cols)

In [ ]:
plot_multiple_lines(x_hat_avgs, x_hat_vars, plot_first_n=25000, legends=[round(x, 2) for x in d], colors=cols)

### Code block for testing implementation

In [ ]:
rng = np.random.default_rng(42)

graph = get_net('complete_mixing', 1000)

start = time.time()
g, ops, comps, y, C, X, op_dif = simulate_deffuant_model(graph, 250000, 0.2, 0.05, symmetric_updating=True, fast_mode=False, rng=rng, offset=-1, res_threshold=0.001)

total_time = round(time.time()-start, 3)
print(f"simulation time: {total_time} seconds in {len(op_dif)}k steps")

### The experiments reported in the paper are over

The rest of the notebook contains first a playground for visualizing the output from the previous code block

Then follows a number of code blocks used for benchmarking our utility functions from the first code block. It also includes other graph generation approaches. We also tried various methods to improve our main graph algorithm for finding connected components but ended up using the functionality provided by graph tool.
Finally there is a very early version of our simulation function built for sparse matrices. However this approach was too inefficient for complete graphs.

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.arange(len(op_dif)), op_dif, linewidth=2.0, color='green')
plt.show()

In [ ]:
plt.hist(ops.a, bins = 50)

In [ ]:
from graph_tool import draw
draw.graph_draw(g)

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.arange(len(X[:])), X[:], linewidth=2.0, color='green')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.arange(len(C[:])), C[:], linewidth=2.0, color='green')
plt.show()

##### Graph generation experiments

In [ ]:
from scipy.sparse import random
from scipy.sparse import tril


def erdos_renyi_scipy(n, p, rng):
    """
    Generates an Erdos-Renyi random graph with n nodes and edge probability p.
    Returns a SciPy sparse matrix representing the adjacency matrix of the graph.
    """
    adj_matrix = random(n, n, density=p, format='csr', dtype="bool", random_state=rng, data_rvs=np.ones)
    adj_matrix = tril(adj_matrix)
    adj_matrix = adj_matrix + adj_matrix.T
    #adj_matrix.data = np.ones_like(adj_matrix.data)
    adj_matrix.setdiag(0)
    return adj_matrix.astype('bool').toarray()

def erdos_renyi(n, p, rng):
    """
    Generates an Erdos-Renyi random graph with n nodes and edge probability p.
    Returns a NumPy array representing the adjacency matrix of the graph.
    """
    adj_matrix = rng.random((n,n), dtype='float32')
    adj_matrix = np.tril(adj_matrix)
    adj_matrix += adj_matrix.T
    adj_matrix[adj_matrix > p] = 0
    adj_matrix[adj_matrix != 0] = 1
    np.fill_diagonal(adj_matrix, 0)
    return adj_matrix.astype("bool")


def erdos_renyi_bin(n, p, rng):
    """
    Generates an Erdos-Renyi graph with n nodes and edge probability p, where self-loops are not allowed.
    :param n: int, number of nodes in the graph
    :param p: float, probability of an edge between any two nodes
    :return: numpy array, adjacency matrix of the graph
    """
    adj_matrix = rng.binomial(1, p, size=(n, n))
    adj_matrix = np.tril(adj_matrix)
    adj_matrix += adj_matrix.T
    np.fill_diagonal(adj_matrix, 0)
    return adj_matrix.astype('bool')






In [ ]:
start = time.time()
ER_sci = erdos_renyi_scipy(10000, p = 0.001, rng = rng)
total_time = round(time.time()-start, 3)
print(f"scipy time: {total_time} seconds")

start = time.time()
ER_num = erdos_renyi_graph(10000, p = 0.001, rng = rng)
total_time = round(time.time()-start, 3)
print(f"numpy time: {total_time} seconds")

start = time.time()
ER_num = erdos_renyi(10000, p = 0.001, rng = rng)
total_time = round(time.time()-start, 3)
print(f"numpy time: {total_time} seconds")

print(f"Scipy size = {ER_sci.data.nbytes} \n Numpy size = {ER_num.nbytes}")

In [ ]:
N = 10000

start = time.time()

test = erdos_renyi_scipy(N, 0.001, rng=rng)
e_list = adj_matrix_to_edge_list(test)
g = Graph(directed=False)
g.add_edge_list(e_list)
total_time = round(time.time()-start, 3)
print(f"scipy time: {total_time} seconds")

start = time.time()
test = erdos_renyi_bin(N, 0.001, rng=rng)
e_list = adj_matrix_to_edge_list(test)
g = Graph(directed=False)
g.add_edge_list(e_list)
total_time = round(time.time()-start, 3)
print(f"from binomial time: {total_time} seconds")


In [ ]:
import networkit as nk
import sys
from graph_tool.generation import complete_graph
from graph_tool import Graph
from graph_tool.generation import random_graph

import time

N = 10000

start = time.time()
test = erdos_renyi_scipy(N, 0.001, rng=rng)
e_list = adj_matrix_to_edge_list(test)
g_ = Graph(directed=False)
g_.add_edge_list(e_list)
total_time = round(time.time()-start, 3)
print(f"numpy time: {total_time} seconds")

def erdos_sample(n, p):
    return (np.random.binomial(n, p))

start = time.time()
g = random_graph(N, lambda: erdos_sample(N, 0.1), directed = False)
total_time = round(time.time()-start, 3)
print(f"numpy time: {total_time} seconds")


In [ ]:
N = 5000

start = time.time()
test = complete_adj(N)
e_list = adj_matrix_to_edge_list(test)
g = Graph(directed=False)
g.add_edge_list(e_list)
total_time = round(time.time()-start, 3)
print(f"gt time: {total_time} seconds")

start = time.time()
g = complete_graph(N)
total_time = round(time.time()-start, 3)
print(f"gt time: {total_time} seconds")

In [ ]:
from graph_tool.topology import label_components

N=10000

g_ = complete_graph(N)

start = time.time()
_, comp = label_components(g_)
total_time = round(time.time()-start, 3)
print(f"gt time: {total_time} seconds")


In [ ]:
from numba import jit

def Reciprocity(adjmatrix):
    """Computes the fraction of reciprocal links to total number of links.
    Both weighted and unweighted input matrices are permitted. Weights
    are ignored for the calculation.
    Parameters
    ----------
    adjmatrix : ndarray of rank-2
        The adjacency matrix of the network.
    Returns
    -------
    reciprocity : float
        A scalar value between 0 (for acyclic directed networks) and 1 (for
        fully reciprocal).
    """
    # 0) PREPARE FOR COMPUTATIONS
    adjmatrix = adjmatrix.astype('bool')

    # 1) COMPUTE THE RECIPROCITY
    L = adjmatrix.sum()
    if L == 0:
        reciprocity = 0
    else:
        # Find the assymmetric links
        # Rest = np.abs(adjmatrix - adjmatrix.T)
        Rest = np.abs(adjmatrix ^ adjmatrix.T)
        Lsingle = 0.5*Rest.sum()
        reciprocity = np.float(L-Lsingle) / L

    return reciprocity

def FloydWarshall_Numba(adjmatrix, weighted_dist=False):
    """Computes the pathlength between all pairs of nodes in a network.
    WARNING! This version returns the same output as 'FloydWarshall()'
        function in main metrics.py module but runs much faster (for networks
        of N > 100). It requires package Numba to be installed.
    Parameters
    ----------
    adjmatrix : ndarray of rank-2
        The adjacency matrix of the network.
    weighted_dist : boolean, optional
        If True, if the graph distances are computed considering the weights
        of the links. False, otherwise. If 'adjmatrix' is a weighted
        network but'weighted_dist = False', the weights of the links are
        ignored.
    Returns
    -------
    distmatrix : ndarray of rank-2
        The pairwise distance matrix dij of the shortest pathlength between
        nodes i and j.
    See Also
    --------
    FloydWarshall : Computes the pathlength between all pairs of nodes.
    """
    # 0) DEFINE THE CORE OF THE FW ALGORITHM, ACCELARATED BY 'Numba'
    @jit
    def FW_Undirected(distmatrix):
        """The Floyd-Warshall algorithm for undirected networks
        """
        N = len(distmatrix)
        for k in range(N):
            for i in range(N):
                for j in range(i,N):
                    d = distmatrix[i,k] + distmatrix[k,j]
                    if distmatrix[i,j] > d:
                        distmatrix[i,j] = d
                        distmatrix[j,i] = d

    @jit
    def FW_Directed(distmatrix):
        """The Floyd-Warshall algorithm for directed networks
        """
        N = len(distmatrix)
        for k in range(N):
            for i in range(N):
                for j in range(N):
                    d = distmatrix[i,k] + distmatrix[k,j]
                    if distmatrix[i,j] > d:
                        distmatrix[i,j] = d

    ########################################################################
    # 1) PREPARE FOR THE CALCULATIONS
    # 1.1) Initialize the distance matrix
    if weighted_dist:
        distmatrix = np.where(adjmatrix == 0, np.inf, adjmatrix)
    else:
        distmatrix = np.where(adjmatrix == 0, np.inf, 1)

    # 1.2) Find out whether the network is directed or undirected
    recip = Reciprocity(adjmatrix)

    # 2) RUN THE FLOYD-WARSHALL ALGORITHM USING FASTER FUNCTIONS (NUMBA)
    if recip==1.0:
        FW_Undirected(distmatrix)
    else:
        FW_Directed(distmatrix)

    return distmatrix

def ConnectedComponents(distmatrix, directed=False, showall=True):
    """Finds all the connected components in a network out of a distance
    matrix.
    A strongly connected component is a set of nodes for which there is
    at least one path connecting every two nodes within the set.
    The function works both for directed and undirected networks, provided
    the adequate distance matrix is given.
    Parameters
    ----------
    distmatrix : ndarray of rank-2
        The pairwise graph distance matrix of the network, usually the
        output of function FloydWarshall().
    directed : boolean, optional
        'True' if the network is directed, 'False' if it is undirected.
    showall : boolean, optional
        If 'True' the function returns all strong components, including
        independent nodes. If 'False' it returns only components of two
        or more nodes.
    Returns
    -------
    components : list
        A list containing the components as ordered lists of nodes.
    See Also
    --------
    FloydWarshall : Pairwise graph distance between all nodes of a network.
    """
    N = len(distmatrix)

    # 1) Detect nodes that are connected in both directions
    newmatrix = np.where(distmatrix < N, 1, 0)

    # If network is directed, consider only pairs with a reciprocal path
    if directed:
        newmatrix = newmatrix * newmatrix.T

    # 2) Sort the nodes into their components
    # nodelist = range(N)
    nodelist = np.arange(N).tolist()
    components = []
    while nodelist:
        # Take the first node. This helps keeping the output sorted
        node = nodelist[0]
        if newmatrix[node,node]:
            # Find the component to which the node belongs to
            comp = list(newmatrix[node].nonzero()[0])
            components.append(comp)
            # Remove nodes in comp from nodelist
            for neigh in comp:
                nodelist.remove(neigh)
            # Clean trash
            del comp
        else:
            # The node is independent. Remove from list and continue
            if showall:
                components.append([node])
            nodelist.remove(node)
            continue

    del newmatrix
    return components

### Following code is a sparse matrix implementation

In [ ]:
import networkx as nx

def get_random_node_uniform(rng, a_huge_key_list):
    L = len(a_huge_key_list)
    i = rng.integers(0, L)
    return a_huge_key_list[i]

def get_net(net_type, net_order, rng=None):
    if net_type == 'scale_free':
        return nx.barabasi_albert_graph(n=net_order, m=2, seed=rng, initial_graph=nx.complete_graph(3))

    elif net_type == 'complete_mixing':
        return nx.complete_graph(n=net_order)

    elif net_type == 'lattice':
        return nx.grid_2d_graph(int(np.sqrt(net_order)), int(np.sqrt(net_order)), periodic=False)

    else:
        return None


def update_opinions(u, v, opinions, convergence, symmetric_updating=True):
    diff = opinions[u] - opinions[v]

    opinions[u] -= convergence * diff
    
    if symmetric_updating:
        opinions[v] += convergence * diff
        
    return opinions


def effective_neighbors(u, adjG, opinions, threshold):
    
    distances = opinions - opinions[u] # vector of opinion distances
    distances[(distances < -threshold) | (distances > threshold)] = -1 # turn nodes at dist>d into -1
    distances[distances != -1] = 0 # the rest into 0
    distances += 1 # close nodes are now 1 and far away ones are 0
    
    return adjG[[u],:]*distances # filters out non-neighbors with close opinion distance


def update_effective_net(u, v, effective_net, adjG, opinions, threshold, symmetric_updating):
    
    neighbors = effective_neighbors(u, adjG, opinions, threshold)

    effective_net[[u],:]   = neighbors
    effective_net[:,[u]] = neighbors.transpose()
        
    if not symmetric_updating:
        return effective_net

    neighbors = effective_neighbors(v, adjG, opinions, threshold)
    
    effective_net[[v],:]   = neighbors
    effective_net[:,[v]] = neighbors.transpose()
            
    return effective_net


def simulate_deffuant_model(G: nx.Graph, iterations: int, threshold: float, convergence: float, initial_opinions=None,\
                            symmetric_updating=True, fast_mode=False, rng=np.random.default_rng(42)):
    """
    Simulates the Deffuant model on the given graph.

    Parameters
    ----------
    G : networkx.Graph
        The graph on which to simulate the model.
    iterations : int
        The number of iterations to run the simulation for.
    threshold : float
        Threshold value for node interaction in the model.
    convergence : float
        The value used for updating opinions in the model.
    symmetric_updating: bool
        If True, at every timestep both nodes update their opinions.
        If False, at every timestep only the first selected node updates its opinion.
    fast_mode: bool
        If True, only the last effective network is returned and used throughout the simulation.
        If False, every effective network is saved and returned.

    Returns
    -------
    list[networkx.Graph]
        A list of graphs, where each graph represents the state of the network at a given time step.
    """
    
    # initialize opinions
    if initial_opinions is None:
        opinions = rng.random(G.order())
    else:
        opinions = np.copy(initial_opinions)
    
    # save adjacency matrix
    adjG = nx.adjacency_matrix(G)

    # get initial effective network
    effective_net = nx.adjacency_matrix(G)
    for u in G.nodes():
        # get 0-1 vector of neighbors at opinion distance <threshold
        neighbors = effective_neighbors(u, adjG, opinions, threshold)
        effective_net[[u],:]   = neighbors
        effective_net[:,[u]] = neighbors.transpose()
    if not fast_mode:
        # initialize the list of nets that will be returned
        network_history = [effective_net]
        opinion_history = [opinions]

    # run simulation
    for t in range(1, iterations):
        
        if not fast_mode:
            # copy last effective network
            effective_net = np.copy(network_history[t-1])
            opinions      = np.copy(opinion_history[t-1])

        # choose random node to update
        u = rng.choice(G.order())
        
        # get another node if u is isolated
        while not effective_net[[u],:].sum():
            u = rng.choice(G.order())
        
        # get a random neighbor of u
        v = get_random_node_uniform(rng, np.argwhere(effective_net[[u]:]==1))[0]

        # update their opinions
        opinions = update_opinions(u, v, opinions, convergence, symmetric_updating)
        
        # we now update the connections of these nodes in the effective network depending on
        # the opinions of their neighbors in the underlying network G
        effective_net = update_effective_net(u, v, effective_net, adjG, opinions, threshold, symmetric_updating)
        
        if not fast_mode:
            # add it to list
            network_history.append(effective_net)
            opinion_history.append(opinions)
    
    if fast_mode:
        return effective_net, opinions
    else:
        return network_history, opinion_history